# Matching Compositions to Recordings 2

Date Started: 2/9/19

It's been a minute since I've tried working on this project, but I do have a large number of new tracks that I've pulled IDs for from Spotify's API. This notebook will look at those tracks, and match them against my listing of ASCAP compositions to see if there's enough matches to move forward with a MVP


**NOTE (1/2/20)**: I'm coming back to this to reference data cleaning code, and to create a script that will do this matching automatically next time. File location strings may be changed in order to properly load data, etc

In [1]:
import json
import re
import sys

import numpy as np
import pandas as pd

## Table of Contents

1. [Bringing in Datasets](#1)
2. [Cleaning Datasets](#2)
3. [Merging Datasets](#3)

<a name="1"></a>
## 1. Bringing in Datasets

In [2]:
comp_artists = pd.read_csv('/home/jovyan/Song_Index/data/comp_artists.csv', index_col=0)
compositions = pd.read_csv('/home/jovyan/Song_Index/data/compositions.csv', index_col=0)
artist_comp_lookup = pd.read_csv('/home/jovyan/Song_Index/data/artist_comp_lookup.csv', index_col=0)
comp_alt_titles = pd.read_csv('/home/jovyan/Song_Index/data/comp_alt_titles.csv', index_col=0)
track_list = pd.read_csv('/home/jovyan/Song_Index/data/main_wfeats.csv', index_col=0, usecols=[0,1,4,8])

### 1a. New Tracks from Last Major Pull

In [3]:
with open('/home/jovyan/SWI_data/data/new_tracks_20190103.json', 'r') as f:
    new_tracks = json.load(f)
    
new_tracks = pd.DataFrame.from_dict(new_tracks, orient='index').reset_index()\
                .rename(columns={'index':'song_id','Song Title': 'song_title',
                                 'Artist': 'artist_name'})

### 1b. Merging Old and New Tracklist

There's roughly 7k duplicates (by `song_id`) b/w the two sets.

In [4]:
# join operation errors out for differing dtypes on each index (song_id) column

track_list = pd.concat([track_list, new_tracks], ignore_index=True, sort=True)\
                        .drop_duplicates(subset='song_id')

In [5]:
track_list.shape

(659315, 3)

### 1c. Readying Compositions Table

In [6]:
def merging_comps(comps, alt_titles, ac_lookup, c_artists):
    '''
    Creates master compositions table with a unique record for each song title and
    alternate titles. Return listing should be quite large.
    
    Each of the required arguments are tables of composition level information 
    originally pulled from ASCAP's website.
    
    Parameters
    ----------
    comps: df, compositions table 
    alt_titles: df, composition alternate titles 
    ac_lookup: df, artist-composition lookup table
    c_artists: df, artist level composition data
    '''
    comp_titles = comps[['CID', 'Title']]
    c_lookup = comps.drop('Title', 1)
    
    at = alt_titles.rename(columns={'alt-title': 'Title'})
    all_comp_titles = pd.concat([comp_titles, at], axis=0, ignore_index=True, sort=True)\
                                .sort_values('CID', 0).dropna(axis=0)
    
    full_comps = pd.merge(c_lookup, all_comp_titles, on='CID')
    print('Titles merged...Now merging in artist names')
   
    full_comps = pd.merge(ac_lookup, all_comp_titles, on='CID')
    full_comps = pd.merge(full_comps, c_artists, on='PID')
    
    return full_comps

In [7]:
comp_list = merging_comps(compositions, comp_alt_titles, artist_comp_lookup, comp_artists)

Titles merged...Now merging in artist names


In [8]:
comp_list.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2500311 entries, 0 to 2500310
Data columns (total 4 columns):
CID               int64
PID               int64
Title             object
Performer Name    object
dtypes: int64(2), object(2)
memory usage: 95.4+ MB


In [9]:
comp_list.head()

,CID,PID,Title,Performer Name
0,0,0,FOR THA LOVE OF MONEY,BONE
1,0,0,FOE THA LOVE OF $ (FEAT. EAZY-E),BONE
2,0,0,FOE THA LOVE OF $ [EXPLICIT],BONE
3,0,0,FOE THA LOVE OF MONEY,BONE
4,0,0,FOE THE LOVE OF MONEY,BONE


<a name="2"></a>
## 2. Cleaning Datasets

### 2a. Standardizing Composition & Track Tables

In [10]:
comp_list['Performer Name'] = comp_list['Performer Name'].iloc[:].apply(lambda x: str(x).lower())
comp_list['Title'] = comp_list['Title'].iloc[:].apply(lambda x: str(x).lower())

track_list['artist_name'] = track_list['artist_name'].iloc[:].apply(lambda x: str(x).lower())\
                                                    .apply(lambda x: str(x).strip("''/*"))
track_list['song_title'] = track_list['song_title'].iloc[:].apply(lambda x: str(x).lower())\
                                                    .apply(lambda x: re.sub(r'(\(feat.*)','', x))

### 2b. Cleaning Track List

In [11]:
cleaning_dict = {
    'single quote mark': r"\"",
    'dbl quote mark': r"\'",
    'brackets': r' \[.*',
    'parenthesis': r'(\s\(.*\))',
    'feat artist': r'( feat\..*)',
    'hyphens' : r' -.*',
}

In [12]:
# removing quotation marks
track_list['artist_name'] = track_list['artist_name'].apply(lambda x: re.sub(r"\"","",x))

# removing parenthesis
track_list['artist_name'] = track_list['artist_name'].apply(lambda x: re.sub(r'(\s\(.*\))', "", x))

# removing feat. artists
track_list['artist_name'] = track_list['artist_name'].apply(lambda x: re.sub(r'( feat\..*)', "", x))

In [13]:
for k in cleaning_dict:
    track_list['song_title'] = track_list['song_title'].iloc[:].apply(
        lambda x: re.sub(cleaning_dict[k],'', x))

### 2d. Cleaning Composition List

In [14]:
for k in cleaning_dict:
    comp_list['Title'] = comp_list['Title'].iloc[:].apply(
        lambda x: re.sub(cleaning_dict[k],'', x))

In [15]:
# removing quotation marks
comp_list['Performer Name'] = comp_list['Performer Name'].apply(lambda x: re.sub(r"\"","",x))

# removing parenthesis
comp_list['Performer Name'] = comp_list['Performer Name'].apply(lambda x: re.sub(r'(\s\(.*\))', "", x))

# removing feat. artists
comp_list['Performer Name'] = comp_list['Performer Name'].apply(lambda x: re.sub(r'( feat\..*)', "", x))

<a name="3"></a>
## 3. Merging Datasets

Merging datasets using the following strategy:

- Lowercasing both song title and artist names for both datasets, also removing quotation marks, brackets, parenthesis, and other misc formatting marks to both artist and song title

In [17]:
test_merge = pd.merge(track_list, comp_list, how='left', left_on=['artist_name', 'song_title'],
                    right_on=['Performer Name', 'Title'])

In [18]:
merged_no_dups = test_merge[test_merge['CID'].notnull()].drop_duplicates(subset='song_id')

In [19]:
len(merged_no_dups)

159437

In [21]:
merged_no_dups.head()

,artist_name,song_id,song_title,CID,PID,Title,Performer Name
0,robyn,6SluaPiV04KOaRTOIScoff,show me love,260151.0,14428.0,show me love,robyn
3,lukas graham,5kqIPrATaCc2LqxVWzQGbk,7 years,43137.0,53876.0,7 years,lukas graham
10,the killers,3aVyHFxRkf8lSjhWdJ68AW,just another girl,76427.0,39793.0,just another girl,the killers
11,tamia,0zIyxS6QxZogHOpGkI6IZH,deeper,8897.0,14338.0,deeper,tamia
14,kanye west,12D0n7hKpPcjuUpcbAKjjr,dont like.1,68936.0,2118.0,dont like.1,kanye west


In [22]:
merged_no_dups.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159437 entries, 0 to 748354
Data columns (total 7 columns):
artist_name       159437 non-null object
song_id           159437 non-null object
song_title        159437 non-null object
CID               159437 non-null float64
PID               159437 non-null float64
Title             159437 non-null object
Performer Name    159437 non-null object
dtypes: float64(2), object(5)
memory usage: 9.7+ MB


In [26]:
matched_songs = merged_no_dups.astype(
    {"CID":"int64", "PID":"int64"})

In [29]:
matched_songs.to_csv('/home/jovyan/Song_Index/data/matched_songs_20200103.csv')